# This notebook ...

In [1]:
import pathlib
import sys
import yaml

import pandas as pd

import torch
import torch.optim as optim
import mlflow
import optuna
import joblib

## Read config

In [2]:
with open(pathlib.Path('.').absolute().parent.parent / "config.yml", "r") as file:
    config = yaml.safe_load(file)

In [3]:
sys.path.append(config['paths']['software_path'])
print(str(pathlib.Path('.').absolute().parent.parent))

## Dataset
from virtual_stain_flow.datasets.PatchDataset import PatchDataset
from virtual_stain_flow.datasets.CachedDataset import CachedDataset

## FNet training
from virtual_stain_flow.models.fnet import FNet
from virtual_stain_flow.trainers.Trainer import Trainer

## wGaN training
from virtual_stain_flow.models.unet import UNet
from virtual_stain_flow.models.discriminator import GlobalDiscriminator
from virtual_stain_flow.trainers.WGaNTrainer import WGaNTrainer

## wGaN losses
from virtual_stain_flow.losses.GradientPenaltyLoss import GradientPenaltyLoss
from virtual_stain_flow.losses.DiscriminatorLoss import DiscriminatorLoss
from virtual_stain_flow.losses.GeneratorLoss import GeneratorLoss

from virtual_stain_flow.transforms.MinMaxNormalize import MinMaxNormalize
from virtual_stain_flow.transforms.PixelDepthTransform import PixelDepthTransform

## Metrics
from virtual_stain_flow.metrics.MetricsWrapper import MetricsWrapper
from virtual_stain_flow.metrics.PSNR import PSNR
from virtual_stain_flow.metrics.SSIM import SSIM

## callback
from virtual_stain_flow.callbacks.MlflowLogger import MlflowLogger
from virtual_stain_flow.callbacks.IntermediatePlot import IntermediatePatchPlot

/home/weishanli/Waylab/pediatric_cancer_atlas_analysis


## Define paths and other train configs

In [4]:
## Loaddata for train
LOADDATA_FILE_PATH = pathlib.Path('.').absolute().parent.parent \
    / '0.data_preprocessing' / 'data_split_loaddata' / 'loaddata_train.csv'
assert LOADDATA_FILE_PATH.exists()
SC_FEATURES_DIR = pathlib.Path(config['paths']['sc_features_path'])

## Output directories
MLFLOW_DIR = pathlib.Path('.').absolute() / 'optuna_mlflow'
MLFLOW_DIR.mkdir(parents=True, exist_ok=True)

OPTUNA_JOBLIB_DIR = pathlib.Path('.').absolute() / 'optuna_joblib'
OPTUNA_JOBLIB_DIR.mkdir(parents=True, exist_ok=True)

## Basic data generation, model convolutional depth and max epoch definition
PATCH_SIZE = 256
DEPTH = 5
EPOCHS = 1_000

## Channels for input and target are read from config
INPUT_CHANNEL_NAMES = config['data']['input_channel_keys']
TARGET_CHANNEL_NAMES = config['data']['target_channel_keys']

## Defines how the train data will be divided to train models of different confluence levels
DATA_GROUPING = {
    'high_confluence': {
        'seeding_density': [12_000, 8_000]
    },
    'low_confluence': {
        'seeding_density': [4_000, 2_000, 1_000]
    }
}

## Define optimization parameters and objectives

In [5]:
import gc
def free_gpu_memory():
    gc.collect()
    torch.cuda.empty_cache()

def objective(trial, dataset, channel_name, confluence_group_name):
    
    ## Suggest hyperparameters
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True) # adam optimizer learning rate
    beta0 = trial.suggest_float('beta0', 0.5, 0.9) # adam optimizer beta 0
    beta1 = trial.suggest_float('beta1', 0.9, 0.999) # adam optimizer beta 1
    betas = (beta0, beta1)

    batch_size = trial.suggest_int('batch_size', 16, 64, step=16)
    patience = trial.suggest_int('patience', 5, 20) # early stop patience

    conv_depth = trial.suggest_int('conv_depth', 3, 5) # convolutional depth for fnet model

    ## Setup model and optimizer
    model = FNet(depth=conv_depth)
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas)
    
    ## Metrics to be computed (and logged)
    metric_fns = {
        "mse_loss": MetricsWrapper(_metric_name='mse', module=torch.nn.MSELoss()),
        "ssim_loss": SSIM(_metric_name="ssim"),
        "psnr_loss": PSNR(_metric_name="psnr"),
    }

    ## Params to log with mlflow
    params = {
            "lr": lr,
            "beta0": beta0,
            "beta1": beta1,
            "depth": conv_depth,
            "patch_size": PATCH_SIZE,
            "batch_size": batch_size,
            "epochs": EPOCHS,
            "patience": patience,
            "channel_name": channel_name,
        }

    ## mlflow logger callback
    mlflow_logger_callback = MlflowLogger(
        name='mlflow_logger',
        mlflow_uri=MLFLOW_DIR / 'mlruns',
        mlflow_experiment_name=f'FNet_optimize_{confluence_group_name}',
        mlflow_start_run_args={'run_name': f'FNet_optimize_{confluence_group_name}_{channel_name}', 'nested': True},
        mlflow_log_params_args=params
    )
    
    ## Trainer
    trainer = Trainer(
        model = model,
        optimizer = optimizer,
        backprop_loss = torch.nn.L1Loss(), # MAE loss for backpropagation
        dataset = dataset,
        batch_size = batch_size,
        epochs = EPOCHS,
        patience = patience,
        callbacks=[mlflow_logger_callback],
        metrics=metric_fns,
        device = 'cuda',
        early_termination_metric='L1Loss'
    )

    # Train the model and log validation loss
    trainer.train()
    val_loss = trainer.best_loss

    del model
    del optimizer
    del metric_fns
    del mlflow_logger_callback
    del trainer
    
    free_gpu_memory()

    return val_loss

In [ ]:
# loaddata_df = pd.read_csv(LOADDATA_FILE_PATH)

# for confluence_group_name, conditions in DATA_GROUPING.items():

#     ## Load dataset
#     loaddata_condition_df = loaddata_df.copy()
#     for condition, values in conditions.items():
#         loaddata_condition_df = loaddata_condition_df[
#             loaddata_condition_df[condition].isin(values)
#         ]

#     sc_features = pd.DataFrame()
#     for plate in loaddata_condition_df['Metadata_Plate'].unique():
#         sc_features_parquet = SC_FEATURES_DIR / f'{plate}_sc_normalized.parquet'
#         if not sc_features_parquet.exists():
#             print(f'{sc_features_parquet} does not exist, skipping...')
#             continue 
#         else:
#             sc_features = pd.concat([
#                 sc_features, 
#                 pd.read_parquet(
#                     sc_features_parquet,
#                     columns=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site', 'Metadata_Cells_Location_Center_X', 'Metadata_Cells_Location_Center_Y']
#                 )
#             ])

#     ## Create patch dataset
#     pds = PatchDataset(
#         _loaddata_csv=loaddata_condition_df,
#         _sc_feature=sc_features,
#         _input_channel_keys=INPUT_CHANNEL_NAMES,
#         _target_channel_keys=TARGET_CHANNEL_NAMES,
#         _input_transform=PixelDepthTransform(src_bit_depth=16, target_bit_depth=8, _always_apply=True),
#         _target_transform=MinMaxNormalize(_normalization_factor=(2 ** 16) - 1, _always_apply=True),
#         patch_size=PATCH_SIZE,
#         verbose=False,
#         patch_generation_method="random_cell",
#         patch_generation_random_seed=42
#     )

#     for channel_name in TARGET_CHANNEL_NAMES:

#         ## Cache dataset of channel
#         pds.set_input_channel_keys(INPUT_CHANNEL_NAMES)
#         pds.set_target_channel_keys(channel_name)
#         cds = CachedDataset(
#             dataset=pds,
#             prefill_cache=True
#         )

#         print(f"Continuing optimization for channel: {channel_name} for {confluence_group_name}")

#         # Load the existing study for the current channel
#         study_path = OPTUNA_JOBLIB_DIR / f"FNet_optimize_{channel_name}_{confluence_group_name}.joblib"
#         if study_path.exists():
#             study = joblib.load(study_path)
#         else:
#             study = optuna.create_study(
#                 direction="minimize",
#                 study_name=f"FNet_optimize_{channel_name}_{confluence_group_name}",
#                 sampler=optuna.samplers.TPESampler(seed=42)
#             )

#         # Optimize the objective function for the current channel
#         study.optimize(lambda trial: objective(trial, cds, channel_name, confluence_group_name), n_trials=50)

#         # Save the updated study for the current channel
#         joblib.dump(study, study_path)

#         # Print best trial results
#         print(f"Best trial for channel {channel_name}:")
#         print(f"  Validation Loss: {study.best_trial.value}")
#         print(f"  Hyperparameters: {study.best_trial.params}")

[I 2025-02-19 16:57:46,573] A new study created in memory with name: FNet_optimize_OrigDNA_high_confluence


Continuing optimization for channel: OrigDNA for high_confluence


2025/02/19 16:57:46 INFO mlflow.tracking.fluent: Experiment with name 'FNet_optimize_high_confluence' does not exist. Creating a new experiment.


Early termination at epoch 123 with best validation metric 0.014454161748290062


[I 2025-02-19 16:59:11,065] Trial 0 finished with value: 0.014454161748290062 and parameters: {'lr': 0.0001329291894316216, 'beta0': 0.8802857225639664, 'beta1': 0.9724674002393291, 'batch_size': 48, 'patience': 7, 'conv_depth': 3}. Best is trial 0 with value: 0.014454161748290062.


...

[I 2025-02-19 16:59:25,285] Trial 2 finished with value: 0.02101070899516344 and parameters: {'lr': 0.00314288089084011, 'beta0': 0.5849356442713105, 'beta1': 0.918000671753503, 'batch_size': 16, 'patience': 9, 'conv_depth': 4}. Best is trial 0 with value: 0.014454161748290062.


Early termination at epoch 56 with best validation metric 0.010364015121012926


[I 2025-02-19 17:00:05,928] Trial 3 finished with value: 0.010364015121012926 and parameters: {'lr': 0.00019762189340280086, 'beta0': 0.6164916560792167, 'beta1': 0.9605734365775156, 'batch_size': 16, 'patience': 9, 'conv_depth': 4}. Best is trial 3 with value: 0.010364015121012926.


Early termination at epoch 112 with best validation metric 0.011310026049613953


[I 2025-02-19 17:01:18,970] Trial 4 finished with value: 0.011310026049613953 and parameters: {'lr': 0.00023345864076016249, 'beta0': 0.8140703845572055, 'beta1': 0.9197677044336776, 'batch_size': 48, 'patience': 14, 'conv_depth': 3}. Best is trial 3 with value: 0.010364015121012926.


In [6]:
RUN_EXTRA = False
DATA_GROUPING = {
    'low_confluence': {
        'seeding_density': [4_000, 2_000, 1_000]
    }
}

loaddata_df = pd.read_csv(LOADDATA_FILE_PATH)

for confluence_group_name, conditions in DATA_GROUPING.items():

    ## Load dataset
    loaddata_condition_df = loaddata_df.copy()
    for condition, values in conditions.items():
        loaddata_condition_df = loaddata_condition_df[
            loaddata_condition_df[condition].isin(values)
        ]

    sc_features = pd.DataFrame()
    for plate in loaddata_condition_df['Metadata_Plate'].unique():
        sc_features_parquet = SC_FEATURES_DIR / f'{plate}_sc_normalized.parquet'
        if not sc_features_parquet.exists():
            print(f'{sc_features_parquet} does not exist, skipping...')
            continue 
        else:
            sc_features = pd.concat([
                sc_features, 
                pd.read_parquet(
                    sc_features_parquet,
                    columns=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site', 'Metadata_Cells_Location_Center_X', 'Metadata_Cells_Location_Center_Y']
                )
            ])

    ## Create patch dataset
    pds = PatchDataset(
        _loaddata_csv=loaddata_condition_df,
        _sc_feature=sc_features,
        _input_channel_keys=INPUT_CHANNEL_NAMES,
        _target_channel_keys=TARGET_CHANNEL_NAMES,
        _input_transform=PixelDepthTransform(src_bit_depth=16, target_bit_depth=8, _always_apply=True),
        _target_transform=MinMaxNormalize(_normalization_factor=(2 ** 16) - 1, _always_apply=True),
        patch_size=PATCH_SIZE,
        verbose=False,
        patch_generation_method="random_cell",
        patch_generation_random_seed=42
    )

    for channel_name in TARGET_CHANNEL_NAMES:

        ## Cache dataset of channel
        pds.set_input_channel_keys(INPUT_CHANNEL_NAMES)
        pds.set_target_channel_keys(channel_name)
        cds = CachedDataset(
            dataset=pds,
            prefill_cache=True
        )

        print(f"Beginning optimization for channel: {channel_name} for {confluence_group_name}")

        # Load the existing study for the current channel
        study_path = OPTUNA_JOBLIB_DIR / f"FNet_optimize_{channel_name}_{confluence_group_name}.joblib"
        if study_path.exists():
            if RUN_EXTRA:
                study = joblib.load(study_path)
            else:
                print("Skipping optimization due to existing joblib...")
                continue
        else:
            study = optuna.create_study(
                direction="minimize",
                study_name=f"FNet_optimize_{channel_name}_{confluence_group_name}",
                sampler=optuna.samplers.TPESampler(seed=42)
            )

        # Optimize the objective function for the current channel
        study.optimize(lambda trial: objective(trial, cds, channel_name, confluence_group_name), n_trials=50)

        # Save the updated study for the current channel
        joblib.dump(study, study_path)

        # Print best trial results
        print(f"Best trial for channel {channel_name}:")
        print(f"  Validation Loss: {study.best_trial.value}")
        print(f"  Hyperparameters: {study.best_trial.params}")

Continuing optimization for channel: OrigDNA for low_confluence
Skipping optimization due to existing joblib...
Continuing optimization for channel: OrigER for low_confluence
Skipping optimization due to existing joblib...


[I 2025-02-20 10:00:57,491] A new study created in memory with name: FNet_optimize_OrigAGP_low_confluence


Continuing optimization for channel: OrigAGP for low_confluence
Early termination at epoch 8 with best validation metric 0.31552654504776


[I 2025-02-20 10:01:06,909] Trial 0 finished with value: 0.31552654504776 and parameters: {'lr': 0.0001329291894316216, 'beta0': 0.8802857225639664, 'beta1': 0.9724674002393291, 'batch_size': 48, 'patience': 7, 'conv_depth': 3}. Best is trial 0 with value: 0.31552654504776.


Early termination at epoch 6 with best validation metric 0.4338683784008026


...

Early termination at epoch 69 with best validation metric 0.0022818949849655232


[I 2025-02-20 11:56:42,772] Trial 35 finished with value: 0.0022818949849655232 and parameters: {'lr': 0.00014261809207046308, 'beta0': 0.8390997896749632, 'beta1': 0.9402514070228026, 'batch_size': 16, 'patience': 8, 'conv_depth': 5}. Best is trial 14 with value: 0.0016418180894106627.


Early termination at epoch 171 with best validation metric 0.0018819176669542987


[I 2025-02-20 12:00:27,755] Trial 36 finished with value: 0.0018819176669542987 and parameters: {'lr': 5.381442180251019e-05, 'beta0': 0.793107694238255, 'beta1': 0.9572731406185142, 'batch_size': 16, 'patience': 19, 'conv_depth': 5}. Best is trial 14 with value: 0.0016418180894106627.
[W 2025-02-20 12:01:36,183] Trial 37 failed with parameters: {'lr': 1.0491515051214714e-05, 'beta0': 0.6239931546649639, 'beta1': 0.9727399654147662, 'batch_size': 16, 'patience': 16, 'conv_depth': 4} because of the following error: Exception("'/home/weishanli/Waylab/pediatric_cancer_atlas_analysis/1.model_optimization/1.2.optimize_fnet_by_confluence/optuna_mlflow/mlruns' does not exist.").
Traceback (most recent call last):
  File "/home/weishanli/anaconda3/envs/speckle_analysis/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_3099078/2534935189.py", line 76, in <lambda>
    study.o

Exception: '/home/weishanli/Waylab/pediatric_cancer_atlas_analysis/1.model_optimization/1.2.optimize_fnet_by_confluence/optuna_mlflow/mlruns' does not exist.